In [1]:
print("Moin!")

Moin!


In [2]:
import os
import json
from collections import defaultdict

import torch
import clip
from PIL import Image
from tqdm import tqdm

In [3]:
import os

current_dir = os.getcwd()
print("Contents of the current working directory: ", current_dir)
for item in os.listdir(current_dir):
    print(item)

Contents of the current working directory:  /home/franzi/mypython/CLIP_img_obj_detect
data
old
.gitignore
test.py
.git
boundingbox.ipynb
.venv
test.ipynb


## IMG to BOX mapping

In [4]:
import os
import json

current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
images_dir = os.path.join(data_dir, 'img')
bboxes_dir = os.path.join(data_dir, 'box')

# Initialize the mapping dictionary
image_to_bbox = {}

# List and sort image files to ensure consistent order
image_files = sorted(f for f in os.listdir(images_dir) if f.endswith('.png'))

for image_file in image_files:
    # Extract image number from filename (e.g., 'img__00001_.png' -> 1)
    try:
        image_num = int(image_file.split('__')[1].split('_')[0])
    except (IndexError, ValueError):
        print(f"Skipping unrecognized image file format: {image_file}")
        continue

    # Adjust for off-by-one indexing
    bbox_num = image_num - 1
    bbox_file = f"bounding_boxes_{bbox_num:05d}.json"

    # Build absolute or relative paths as needed
    image_path = os.path.join('data/images', image_file)
    bbox_path = os.path.join('data/bounding_boxes', bbox_file)

    # Add to mapping
    image_to_bbox[image_path] = bbox_path

imagedirmap = {}
# Optional: Print or save mapping
print("Image to bounding box mapping:")
for img, bbox in image_to_bbox.items():
    # print(f"{img} --> {bbox}")
    imagedirmap[img] = bbox

imagedirmap
# Optional: Save to JSON
# with open('image_bbox_mapping.json', 'w') as f:
#     json.dump(image_to_bbox, f, indent=2)


Image to bounding box mapping:


{'data/images/img__00001_.png': 'data/bounding_boxes/bounding_boxes_00000.json',
 'data/images/img__00002_.png': 'data/bounding_boxes/bounding_boxes_00001.json',
 'data/images/img__00003_.png': 'data/bounding_boxes/bounding_boxes_00002.json',
 'data/images/img__00004_.png': 'data/bounding_boxes/bounding_boxes_00003.json',
 'data/images/img__00005_.png': 'data/bounding_boxes/bounding_boxes_00004.json',
 'data/images/img__00006_.png': 'data/bounding_boxes/bounding_boxes_00005.json',
 'data/images/img__00007_.png': 'data/bounding_boxes/bounding_boxes_00006.json',
 'data/images/img__00008_.png': 'data/bounding_boxes/bounding_boxes_00007.json',
 'data/images/img__00009_.png': 'data/bounding_boxes/bounding_boxes_00008.json',
 'data/images/img__00010_.png': 'data/bounding_boxes/bounding_boxes_00009.json',
 'data/images/img__00011_.png': 'data/bounding_boxes/bounding_boxes_00010.json',
 'data/images/img__00012_.png': 'data/bounding_boxes/bounding_boxes_00011.json',
 'data/images/img__00013_.pn

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "ViT-B/32"
SIMILARITY_THRESHOLD = 0.15  # tune this on a small validation set

DEVICE

'cpu'

In [6]:
model, preprocess = clip.load(MODEL_NAME, device=DEVICE)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          